In [5]:
import time
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torchvision
import os
from torchvision import datasets, transforms
from torch.nn.utils.rnn import *
from phoneme_list import *
import torch.nn.functional as F
import Levenshtein as lev
from ctcdecode import CTCBeamDecoder

In [2]:
!nvidia-smi

Tue Nov 10 15:56:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   39C    P2    68W / 280W |    984MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 70%   87C    P2   234W / 280W |  23526MiB / 24220MiB |     95%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [ ]:
!kill 3911  

In [3]:
trainset =np.load('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/hw3p2/train.npy',allow_pickle=True)
validset=np.load('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/hw3p2/dev.npy',allow_pickle=True)
testset=np.load('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/hw3p2/test.npy',allow_pickle=True)

In [4]:
train_labels=np.load('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/hw3p2/train_labels.npy',allow_pickle=True)
valid_labels=np.load('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/hw3p2/dev_labels.npy',allow_pickle=True)

In [6]:
def my_collate(args): 
    X=[]
    X_len=[]
    Y=[]
    Y_len=[]
    for i in args:        
        X.append(i[0])
        X_len.append(i[1])
        Y.append(i[2])
        Y_len.append(i[3])
        
    X_pad=pad_sequence(X)
    Y_pad=pad_sequence(Y,batch_first=True)
    
    return X_pad,X_len,Y_pad,Y_len 

In [7]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,X,Y):        
        self.X=X    
        self.Y=Y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,index): 

        X=self.X[index]
        Y=self.Y[index]+1#For dealing with blank in CTC loss
        
        return torch.from_numpy(X),X.shape[0],torch.from_numpy(Y),Y.shape[0]

In [8]:
train_dataset=MyDataset(trainset,train_labels)
valid_dataset=MyDataset(validset,valid_labels)
train_loader=torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=32,num_workers=36,collate_fn=my_collate,pin_memory=True,)
valid_loader=torch.utils.data.DataLoader(valid_dataset,shuffle=False,batch_size=32,num_workers=36,collate_fn=my_collate,pin_memory=True)

In [9]:
train_dataset.__getitem__(1)

(tensor([[ 74.5233, -25.8695, -18.9476,  ...,  10.0848,  14.3440,  -1.3440],
         [ 73.6109, -28.4520, -21.8628,  ...,  -1.8643,  12.6011,  -2.1498],
         [ 72.0141, -23.9326, -16.5214,  ...,  -0.0897,  20.6820,  -0.4232],
         ...,
         [ 70.6455, -23.2869, -17.0041,  ..., -12.4953,  17.7580,  -4.2217],
         [ 71.2157, -23.9326, -20.4052,  ..., -12.0009,  21.0475,   9.4115],
         [ 71.5579, -23.9326, -22.8345,  ...,   5.2342,  25.7990,  16.3243]],
        dtype=torch.float64),
 1592,
 tensor([ 1, 12,  5, 33, 18,  4, 11, 19, 33, 31,  6, 30, 31,  5, 38, 15,  9,  4,
         22, 19, 25, 12,  5, 38, 35, 11, 40,  5, 37, 12, 20, 27, 23, 11, 22,  5,
         34,  9, 14, 33, 29, 23, 15, 31,  1, 19, 33, 38,  5, 40, 30, 19, 29, 39,
         36, 33, 19, 11, 33, 19,  9, 20,  5, 25, 19, 25, 33, 30,  5, 22,  5, 33,
         18, 13, 11, 23,  6, 26,  9, 30, 35, 22, 19, 25, 19, 33, 31,  1, 14, 23,
         20, 14, 22,  6, 30, 31, 34, 30, 36, 12, 27, 40, 38, 35, 11, 40,  1, 38,


In [10]:
class Model(nn.Module):
    def __init__(self, in_size, out_size, hidden_size):
        super(Model, self).__init__()
        
        out_channel1=512
        out_channel2=512
        
        self.convd1=nn.Conv1d(in_size,out_channel1,kernel_size=3, padding=1,bias=True)
        self.bn1 = nn.BatchNorm1d(out_channel1)
        self.convd2=nn.Conv1d(out_channel1,out_channel2,kernel_size=3, padding=1,bias=True)
        self.bn2 = nn.BatchNorm1d(out_channel2)
        self.lstm = nn.LSTM(out_channel2,hidden_size,num_layers=4,bidirectional=True)
        self.l1 = nn.Linear(hidden_size * 2, 256)
        self.l2 = nn.Linear(256, out_size)
    
    def forward(self, X, lengths):
        
        X=X.permute(1,2,0)
        X=self.convd1(X)
        X=F.relu(self.bn1(X))
        X=self.convd2(X)
        X=F.relu(self.bn2(X))
        X=X.permute(2,0,1)
        
        packed_X = pack_padded_sequence(X, lengths, enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        out, out_lens = pad_packed_sequence(packed_out)
        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        
        out=self.l2(F.relu(self.l1(out)))
        out = out.log_softmax(2)
        
        return out, out_lens

In [10]:
#Use only when we need to train from scratch
model = Model(13, 42 ,512)
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = nn.DataParallel(model,device_ids=[0]).to(device)
model

DataParallel(
  (module): Model(
    (convd1): Conv1d(13, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (convd2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (lstm): LSTM(512, 512, num_layers=4, bidirectional=True)
    (l1): Linear(in_features=1024, out_features=256, bias=True)
    (l2): Linear(in_features=256, out_features=42, bias=True)
  )
)

In [11]:
model=Model(13, 42 ,512)
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load('Model_final_kaggle.pth')#checkpoint
model = nn.DataParallel(model,device_ids=[0]).to(device)
model.load_state_dict(checkpoint)
model

DataParallel(
  (module): Model(
    (convd1): Conv1d(13, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (convd2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (lstm): LSTM(512, 512, num_layers=4, bidirectional=True)
    (l1): Linear(in_features=1024, out_features=256, bias=True)
    (l2): Linear(in_features=256, out_features=42, bias=True)
  )
)

In [12]:
#We load the model onto CUDA
criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma=0.85)
num_epochs=30

In [13]:
def LevScore(pred, target):
    return lev.distance(pred.replace(" ", ""), target.replace(" ", ""))

In [14]:
def evaluate(data_loader):
    model.eval()
    decoder = CTCBeamDecoder(['$'] * len(PHONEME_LIST), beam_width=20, blank_id=0, log_probs_input=True)#included blank id as per Eason
    
    valid_loss=0
    dict_phoneme={}
    total_currCharErr=0
    for i in range(len(PHONEME_MAP)):
            dict_phoneme[i]=PHONEME_MAP[i]
            
    def list_return(list_any,dict_phoneme):
        string_final=[]
        for i in list_any:
            #We have to map them
            i=i.numpy()
            k=''
            for ele in i:
                k+=dict_phoneme[ele]    
            string_final.append(k) 
        return string_final
          
    for batch_idx,(x,x_len,y,y_len) in enumerate(data_loader):   
        list_best=[]
        x=x.float().to(device)
        y=y.to(device)
        
        output,out_lens=model(x,x_len)
        y_len=torch.from_numpy(np.array(y_len, dtype=np.int))
        loss=criterion(output,y,out_lens,y_len)
        valid_loss+=loss.item() 
        test_Y, _, _, test_Y_lens = decoder.decode(output.transpose(0, 1), out_lens)
        
        for i in range(test_Y.size(0)):
            list_best.append(test_Y[i,0,:test_Y_lens[i, 0]]) 
        #print("batch :{} ,Valid loss  :{}".format(batch_idx+1,valid_loss/(batch_idx+1)))
            
        pred=list_return(list_best,dict_phoneme)
        #print("length of pred_list :",len(pred))
        target=list_return(y.cpu(),dict_phoneme)
        #print("length of target_list :",len(target))
        currCharErr=0
        for i in range(0,len(pred)):
            currCharErr= LevScore(pred[i],target[i])
        #print("Distance error :",currCharErr)
        #print("batch :{} ,Distance error :{}".format(batch_idx+1,currCharErr))
        total_currCharErr+= currCharErr           
            
    dist_loss = total_currCharErr / len(data_loader)
    valid_loss_total=valid_loss / len(data_loader)
    model.train()
    return valid_loss, dist_loss

In [15]:
def train(num_epochs,train_loader,valid_loader):
    
    for epoch in range(num_epochs):
        model.train()
        train_loss=0
        print("epoch :",epoch+1)
        for batch_idx,(x,x_len,y,y_len) in enumerate(train_loader):
            start=time.time()
            x=x.float().to(device)
            y=y.to(device)

            output,out_lens=model(x,x_len)
            y_len=torch.from_numpy(np.array(y_len, dtype=np.int))
            loss=criterion(output,y,out_lens,y_len)
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()

            train_loss+=loss.item()
            
            stop=time.time()
            print('E: %d, B: %d / %d, Train Loss: %.3f | avg_loss: %.3f, Time Taken : %.3f' % (epoch+1, batch_idx+1, 
                                                                           len(train_loader),
                  loss.item(),train_loss/(batch_idx+1),stop-start),end='\n ')
            
            if epoch%5==0:
                torch.save(model.state_dict(), "Model_final_1.pth")
            del loss
            del y
            del x
            
        valid_loss, dist_loss = evaluate(valid_loader)
        print('Distance loss: ',dist_loss)
        if dist_loss<=6.5:
            torch.save(model.state_dict(), "Model_final_kaggle.pth")
            break
        #scheduler.step(valid_loss)
        scheduler.step()

In [27]:
'''
Added dropput to LSTM
DataParallel(
  (module): Model(
    (convd1): Conv1d(13, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (convd2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (lstm): LSTM(512, 512, num_layers=4, bidirectional=True)
    (l1): Linear(in_features=1024, out_features=256, bias=True)
    (l2): Linear(in_features=256, out_features=42, bias=True)
  )
)
'''
train(num_epochs,train_loader,valid_loader)

epoch : 1
E: 1, B: 1 / 688, Train Loss: 0.090 | avg_loss: 0.090, Time Taken : 1.468
 E: 1, B: 2 / 688, Train Loss: 0.058 | avg_loss: 0.074, Time Taken : 1.345
 E: 1, B: 3 / 688, Train Loss: 0.073 | avg_loss: 0.074, Time Taken : 1.334
 E: 1, B: 4 / 688, Train Loss: 0.078 | avg_loss: 0.075, Time Taken : 1.300
 E: 1, B: 5 / 688, Train Loss: 0.069 | avg_loss: 0.074, Time Taken : 1.347
 E: 1, B: 6 / 688, Train Loss: 0.082 | avg_loss: 0.075, Time Taken : 1.303
 E: 1, B: 7 / 688, Train Loss: 0.059 | avg_loss: 0.073, Time Taken : 1.288
 E: 1, B: 8 / 688, Train Loss: 0.070 | avg_loss: 0.072, Time Taken : 1.316
 E: 1, B: 9 / 688, Train Loss: 0.073 | avg_loss: 0.072, Time Taken : 1.265
 E: 1, B: 10 / 688, Train Loss: 0.071 | avg_loss: 0.072, Time Taken : 1.322
 E: 1, B: 11 / 688, Train Loss: 0.066 | avg_loss: 0.072, Time Taken : 1.285
 E: 1, B: 12 / 688, Train Loss: 0.083 | avg_loss: 0.073, Time Taken : 1.292
 E: 1, B: 13 / 688, Train Loss: 0.058 | avg_loss: 0.071, Time Taken : 1.313
 E: 1, B: 14

KeyboardInterrupt: 

In [16]:
model

DataParallel(
  (module): Model(
    (convd1): Conv1d(13, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (convd2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (lstm): LSTM(512, 512, num_layers=4, bidirectional=True)
    (l1): Linear(in_features=1024, out_features=256, bias=True)
    (l2): Linear(in_features=256, out_features=42, bias=True)
  )
)

In [28]:
#We get the test data
def my_collate_test(args): 
    X=[]
    X_len=[]
    for i in args:        
        X.append(i[0])
        X_len.append(i[1])
        
    X_pad=pad_sequence(X)
    
    return X_pad,X_len

class MyDatasettest(torch.utils.data.Dataset):
    def __init__(self,X):        
        self.X=X    
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,index): 

        X=self.X[index]
        
        return torch.from_numpy(X),X.shape[0]

In [29]:
test_dataset=MyDatasettest(testset)
test_loader=torch.utils.data.DataLoader(test_dataset,shuffle=False,batch_size=30,num_workers=36,collate_fn= my_collate_test,pin_memory=True,)

In [30]:
len(PHONEME_LIST)
best_list=[]

In [31]:
decoder = CTCBeamDecoder(['$'] * len(PHONEME_LIST), beam_width=20, blank_id=0, log_probs_input=True)#included blank id as per Eason
list_best=[]
model.eval()
for batch_idx,(x,x_len) in enumerate(test_loader):                                                 
    x=x.float().to(device)
    output,out_lens=model(x,x_len)
    
    test_Y, _, _, test_Y_lens = decoder.decode(output.transpose(0, 1), out_lens)
    for i in range(test_Y.size(0)):
            list_best.append(test_Y[i,0,:test_Y_lens[i, 0]]) 
    
    #list_best.append(test_Y[:,0,:test_Y_lens[:, 0]])
    print('batch:',batch_idx+1)

batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76


In [32]:
test_Y_lens.shape

torch.Size([1, 20])

In [33]:
list_best

[tensor([ 1, 31, 33,  5, 16,  5, 33, 19, 25, 33, 36, 39, 36,  1, 18, 19, 40,  9,
         13, 23, 20, 22,  7, 25, 31,  5, 23, 11, 18, 19, 24,  1],
        dtype=torch.int32),
 tensor([ 1,  4, 16, 33, 14, 14, 23, 20, 25,  8, 33, 16,  6, 23, 12, 20, 39, 13,
         23, 27, 23,  4, 24, 29, 31, 38, 35, 11, 23,  8, 33, 18,  5, 29, 18, 20,
         30,  5, 25, 11, 12, 13, 30, 12,  5, 31, 22, 38,  3, 23,  5, 11, 22, 38,
          6, 30, 33, 14,  5, 37, 12,  5,  9, 30,  6, 16,  5, 23, 40,  1],
        dtype=torch.int32),
 tensor([ 1, 12,  5,  7,  9, 14, 33, 20,  4, 25, 11, 15, 17, 35, 11, 19, 25, 39,
         35, 30, 24,  8, 25, 11,  1], dtype=torch.int32),
 tensor([ 1, 25,  5, 24,  9,  5, 33, 13, 25,  1, 16, 30, 13, 32, 25, 13, 23, 39,
         19, 40, 38, 15, 33, 19, 26,  3, 25, 39, 36,  1, 22, 35, 11, 25,  8, 33,
         18,  5, 40,  9,  5, 25, 11,  1], dtype=torch.int32),
 tensor([ 1, 12,  5, 24, 39, 36, 40, 19, 22, 15, 24, 25, 19, 30, 14,  4, 25, 11,
         18, 20, 30, 19, 22,  6, 23,

In [34]:
dict_phoneme={}
for i in range(len(PHONEME_MAP)):
    dict_phoneme[i]=PHONEME_MAP[i]

In [35]:
#We map to phonomene map
count=0
string_final=[]
for i in list_best:
    #We have to map them
    i=i.numpy()
    k=''
    for ele in i:
        k+=dict_phoneme[ele]    
        #print(k)
        
    string_final.append(k) 
    count+=1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [36]:
len(string_final)

2251

In [37]:
sample_csv=pd.read_csv('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/sample_submission.csv')

#We read in the sample csv and replace its last column with the scores we have achieved
n = sample_csv.columns[1]
sample_csv[n] = string_final

submission_csv=sample_csv

submission_csv.to_csv('/home/ironman/shayeree/records/11-785-fall-20-homework-3/11-785-summer-20-homework-3part-2/submission25.csv',index=False) 